In [6]:
file_paths = [
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/8th-Science.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Math-I.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Math-II.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Physics-I.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Physics-II.csv"
]

In [7]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = pd.read_csv(file_paths[0])

# Display the dataframe
df.head()

,English Question,Bengali Question,A,B,C,D,A Bn,B Bn,C Bn,D Bn,Correct Answer
0,Which acid present in tamarind?,তেঁতুলে কোন এসিড থাকে?,Malic,Citric,Oxalic,Tartaric,ম্যালিক,সাইট্রিক,অক্সালিক,টারটারিক,d
1,By which two elements fuse is made?,কোন মৌল দুটির সাহায্যে ফিউজ প্রস্তুত করা হয়?,Lead and Zinc,Zinc and Tin,Lead and Tin,Lead and Iron,সীসা ও জিঙ্ক,জিঙ্ক ও টিন,সীসা ও টিন,সীসা ও লোহা,c
2,Which on is the tertiary consumer of sundarbans?,সুন্দরবনের তৃতীয় স্তরের খাদক কোনটি?,Pig,Deer,Monkey,Hen,শুকর,হরিণ,বানর,মুরগি,a
3,Which hormone is synthesized in leaves?,পাতায় উৎপন্ন হরমোন কোনটি?,Auxin,Ethylene,Florigen,Gibberellin,অক্সিন,ইথিলিন,ফ্লোরিজেন,জিব্বেরেলিন,c
4,Which one is used as insecticide?,ফসলের ক্ষতিকর পেস্ট দমন করে কোনটি?,NH_{4}OH,NaOH,Mg(OH)_{2},Ca(OH)_{2},NH_{4}OH,NaOH,Mg(OH)_{2},Ca(OH)_{2},d


In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

In [9]:
PROJECT_ID = 'playexch-80b6'

In [10]:
LOCATION = "us-central1"

In [11]:
vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-1.5-flash")

def get_gemini_output(prompt = "", max_out_tokens = 128, temp = 0.0, top_p = 0.9, top_k = 10):
    responses = model.generate_content(
        contents=prompt,
        generation_config={
            "max_output_tokens": max_out_tokens,
            "temperature": temp,
            "top_p": top_p,
            "top_k": top_k
        },
    stream=True,
    )

    response_list = []
    for response in responses:
        try:
            response_list.append(response.text)
        except IndexError:
            response_list.append("")
            continue
    prompt_response = " ".join(response_list)
    return prompt_response

In [12]:
get_gemini_output("বাংলাদেশের ৫ জন কবির নাম লিস্ট আকারে লিখ।")

'বা ংলাদেশের ৫ জন কবির নাম লিস্ট  আকারে:\n\n1. কাজী নজরুল ইস লাম\n2. রবীন্দ্রনাথ ঠাকুর\n3. মাইকেল মধুসূদন দত্ত\n 4. সুকান্ত ভট্টাচার্য\n5. জীবনানন্দ দাশ \n '

In [17]:
zero_shot = """আপনাকে একটি প্রশ্ন এবং চারটি বিকল্প উত্তর দেওয়া হয়েছে। আপনাকে সঠিক উত্তরটি নির্ধারণ করতে হবে এবং শুধুমাত্র ক, খ, গ, ঘ বিকল্পের মধ্যে থেকে সঠিক উত্তরটি লিখতে হবে। 

উত্তরের জন্য প্রথমে অপশন নাম্বারটি লিখে (ক / খ/ গ / ঘ) ব্রাকেট ) দিয়ে উত্তর টি লিখুন। অপশনের মতোই নিম্নোক্ত ফরম্যাটটি অনুসরণ করুন।
ক / খ / গ / ঘ) উত্তর 

"""

In [18]:
three_shot = """আপনাকে একটি প্রশ্ন এবং চারটি বিকল্প উত্তর দেওয়া হয়েছে। আপনাকে সঠিক উত্তরটি নির্ধারণ করতে হবে এবং শুধুমাত্র ক, খ, গ, ঘ বিকল্পের মধ্যে থেকে সঠিক উত্তরটি লিখতে হবে। 

উত্তরের জন্য প্রথমে অপশন নাম্বারটি লিখে (ক / খ/ গ / ঘ) ব্রাকেট ) দিয়ে উত্তর টি লিখুন। অপশনের মতোই নিম্নোক্ত ফরম্যাটটি অনুসরণ করুন।
ক / খ / গ / ঘ) উত্তর 

### উদাহরণ ১
### প্রশ্ন:
p^{2} + q^{2} = 3, pq = 3 হলে (p + q)^{2} এর মান কত?

### বিকল্প:
ক) 3
খ) 9
গ) 11
ঘ) 15

### আপনার উত্তর:
খ) 9

### উদাহরণ ২
### প্রশ্ন:
x - 3y = 15 এবং 3x + y = 3ন রেখাদ্বয়ের ঢালের গুণফল কত?

### বিকল্প:
ক) -3
খ) -1
গ) 1
ঘ) 3

### আপনার উত্তর:
খ) -1

### উদাহরণ ৩
### প্রশ্ন:
বিনা বাধায় পড়ন্ত বস্তু 5 সেকেন্ডে 50 মিটার পথ অতিক্রম করলে 72 মিটার পথ অতিক্রম করতে কত সেকেন্ড সময় লাগবে?

### বিকল্প:
ক) 6
খ) 7.2
গ) 9.5
ঘ) 12

### আপনার উত্তর:
ক) 6

### আপনার কাজ

"""

In [19]:
chain_of_thought = """আপনাকে একটি প্রশ্ন এবং চারটি বিকল্প উত্তর দেওয়া হয়েছে। আপনাকে সঠিক উত্তরটি নির্ধারণ করতে হবে এবং শুধুমাত্র ক, খ, গ, ঘ বিকল্পের মধ্যে থেকে সঠিক উত্তরটি লিখতে হবে। 

উত্তরের জন্য প্রথমে অপশন নাম্বার টি লিখে (ক / খ/ গ / ঘ) ব্রাকেট ) দিয়ে উত্তর টি লিখুন। অপশনের মতোই নিম্নোক্ত ফরম্যাটটি অনুসরণ করুন।
ক / খ / গ / ঘ) উত্তর 

প্রশ্নটির উত্তর নির্ধারণ করতে গিয়ে নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: প্রশ্নটি বোঝা
প্রশ্নটি ভাল করে পড়ুন এবং এটি কী জানতে চাচ্ছে তা বোঝার চেষ্টা করুন। প্রশ্নটির প্রয়োজনীয়তা এবং কনটেক্সটটি বোঝার জন্য এটি বিস্তারিতভাবে বিবেচনা করুন।

### ধাপ ২: বিকল্পগুলি পড়া
প্রতিটি বিকল্প ভাল করে পড়ুন এবং কোন কোন বিকল্পগুলি সম্ভবত সঠিক হতে পারে তা সনাক্ত করুন। প্রতিটি বিকল্পের সাথে প্রশ্নটির সম্পর্ক বিচার করুন।

### ধাপ ৩: বিকল্পগুলি মূল্যায়ন
প্রশ্নটির সাথে সম্পর্কিত জ্ঞান এবং তথ্য ব্যবহার করে সম্ভাব্য সমাধানে পৌছান। কোন বিকল্পটি সঠিক হতে পারে তা বিচার করুন।

### ধাপ ৪: সঠিক উত্তর নির্বাচন
প্রথম তিনটি ধাপের ভিত্তিতে সবচেয়ে যুক্তিযুক্ত এবং সঠিক উত্তরটি নির্বাচন করুন।

### উদাহরণ ১
### প্রশ্ন:
p^{2} + q^{2} = 3, pq = 3 হলে (p + q)^{2} এর মান কত?

### বিকল্প:
ক) 3
খ) 9
গ) 11
ঘ) 15

#### চিন্তাধারা:
- ধাপ ১: প্রশ্নটি বোঝা: এখানে, p^{2} + q^{2} এবং pq এর মান দেওয়া হয়েছে এবং (p + q)^{2} এর মান নির্ধারণ করতে বলা হয়েছে।
- ধাপ ২: বিকল্পগুলি পড়া: বিকল্পগুলি হল ৩, ৯, ১১, এবং ১৫।
- ধাপ ৩: সম্ভাব্য সমাধান: আমরা জানি যে (p + q)^{2} = p^{2} + 2pq + q^{2}। সুতরাং, (p + q)^{2} = 3 + 2*3 = 9।
- ধাপ ৪: সঠিক উত্তর নির্বাচন: সুতরাং, সঠিক উত্তর হল খ) ৯।

### আপনার উত্তর:
খ) ৯

### উদাহরণ ২
### প্রশ্ন:
x - 3y = 15 এবং 3x + y = 3ন রেখাদ্বয়ের ঢালের গুণফল কত?

### বিকল্প:
ক) -3
খ) -1
গ) 1
ঘ) 3

#### চিন্তাধারা:
- ধাপ ১: প্রশ্নটি বোঝা: এখানে, দুইটি রেখার সমীকরণ দেওয়া হয়েছে এবং তাদের ঢালের গুণফল নির্ধারণ করতে বলা হয়েছে।
- ধাপ ২: বিকল্পগুলি পড়া: বিকল্পগুলি হল -৩, -১, ১, এবং ৩।
- ধাপ ৩: সম্ভাব্য সমাধান: আমরা x - 3y = 15 কে y = mx + c আকারে লিখতে পারি: y = 1/3x + 15। ফলে, প্রথম রেখার ঢাল হল ১/৩ এবং দ্বিতীয় রেখার ঢাল হল -৩। সুতরাং, ঢালের গুণফল হল (১/৩) * (-৩) = -১।
- ধাপ ৪: সঠিক উত্তর নির্বাচন: সুতরাং, সঠিক উত্তর হল খ) -১।

### আপনার উত্তর:
খ) -১

### উদাহরণ ৩
### প্রশ্ন:
বিনা বাধায় পড়ন্ত বস্তু 5 সেকেন্ডে 50 মিটার পথ অতিক্রম করলে 72 মিটার পথ অতিক্রম করতে কত সেকেন্ড সময় লাগবে?

### বিকল্প:
ক) 6
খ) 7.2
গ) 9.5
ঘ) 12

#### চিন্তাধারা:
- ধাপ ১: প্রশ্নটি বোঝা: এখানে, বিনা বাধায় পড়ন্ত বস্তুর সময় নির্ধারণ করা।
- ধাপ ২: বিকল্পগুলি পড়া: বিকল্পগুলি হল ৬, ৭.২, ৯.৫, এবং ১২।
- ধাপ ৩: সম্ভাব্য সমাধান: বিনা বাধায় পড়ন্ত বস্তু t সময়ে h দূরত্ব অতিক্রম করে, যেখানে h = 1/2 * g * t^2। 50 মিটার যেতে 5 সেকেন্ড সময় লাগে: 50 = 1/2 * g * 5^2, g = 4 মিটার/সেকেন্ড^2। সুতরাং, 72 মিটার যেতে সময় লাগবে: 72 = 1/2 * 4 * t^2, t^2 = 36, t = 6 সেকেন্ড।
- ধাপ ৪: সঠিক উত্তর নির্বাচন: সুতরাং, সঠিক উত্তর হল ক) ৬।

### আপনার উত্তর:
খ) ৭.২

### আপনার কাজ


"""

### Zero-shot

In [27]:
def prepare_questions(row, base_prompt, prompt_type = 'zero_shot'):
    question = row['Bengali Question']
    options = {
        'ক': row['A Bn'],
        'খ': row['B Bn'],
        'গ': row['C Bn'],
        'ঘ': row['D Bn']
    }
    correct_answer = row['Correct Answer']
    eng2ben_map = {
        'a': 'ক',
        'b': 'খ',
        'c': 'গ',
        'd': 'ঘ'
    }
    correct_option = eng2ben_map[correct_answer] + ") " + options[eng2ben_map[correct_answer]]

    options_string = "\n".join([f"{key}) {value}" for key, value in options.items()])

    if prompt_type == 'zero_shot':
        question_prompt = base_prompt + f"### প্রশ্ন:\n{question}\n\n### বিকল্প:\n{options_string}\n\n### আপনার উত্তর:\n"
    elif prompt_type == 'three_shot':
        question_prompt = base_prompt + f"### প্রশ্ন:\n{question}\n\n### বিকল্প:\n{options_string}\n\n### আপনার উত্তর:\n"
    elif prompt_type == 'cot':
        question_prompt = base_prompt + f"### প্রশ্ন:\n{question}\n\n### বিকল্প:\n{options_string}\n\n"

    return question_prompt, correct_option

In [54]:
list(range(4))

[0, 1, 2, 3]

In [53]:
import time
from tqdm.notebook import tqdm

file_paths = [
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/8th-Science.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Math-I.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Math-II.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Physics-I.csv",
    "https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Physics-II.csv"
]

file_tags = [
    "8th_science",
    "12th_math_1",
    "12th_math_2",
    "12th_physics_1",
    "12th_physics_2"
]

def prepare_zs_results(df, dataset_tag):
    # Assuming df is the original DataFrame and category_translation is defined
    df['predicted_option'] = None
    df['correct_option_fmt'] = None

    # Try to load the progress from the Parquet file
    try:
        df_progress = pd.read_parquet(f"gemini_zs_rqa_{dataset_tag}_results.parquet")
        start_index = df_progress['predicted_option'].last_valid_index() + 1
    except (FileNotFoundError, ValueError):
        df_progress = df.copy()
        start_index = 0


    for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
        
        try:
            prompt_input, cor_opt = prepare_questions(row, base_prompt = zero_shot, prompt_type = 'zero_shot')
        except:
            print("Error w/ sample", row)

        try:
            df.at[i, 'predicted_option'] = get_gemini_output(prompt_input, max_out_tokens = 128)
            df.at[i, 'correct_option_fmt'] = cor_opt
        except Exception as e:
            print(f"Error at index {i}")
            time.sleep(5)

        if (i + 1) % 100 == 0:
            df.to_parquet(f"gemini_zs_rqa_{dataset_tag}_results.parquet")

    # Save final progress
    df.to_parquet(f"gemini_zs_rqa_{dataset_tag}_results.parquet")
    return df

### set file index

for file_index in range(4):
    print(file_paths[file_index])
    df = pd.read_csv(file_paths[file_index])

    print(df.head())

    zs_results = prepare_zs_results(df, dataset_tag = file_tags[file_index])
    print(zs_results.head())

https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/8th-Science.csv
                                   English Question  \
0                   Which acid present in tamarind?   
1               By which two elements fuse is made?   
2  Which on is the tertiary consumer of sundarbans?   
3           Which hormone is sybtgesuzed in leaves?   
4                 Which one is used as insecticide?   

                               Bengali Question              A             B  \
0                        তেঁতুলে কোন এসিড থাকে?          Malic        Citric   
1  কোন মৌল দুটির সাহায্যে ফিউজ প্রস্তুত করা হয়?  Lead and Zinc  Zinc and Tin   
2          সুন্দরবনের তৃতীয় স্তরের খাদক কোনটি?            Pig          Deer   
3                     পাতায় উৎপন্ন হরমোন কোনটি?          Auxin      Ethylene   
4            ফসলের ক্ষতিকর পেস্ট দমন করে কোনটি?       NH_{4}OH          NaOH   

              C              D          A Bn         B Bn        C Bn  \
0        Oxalic       Tartaric       

  0%|          | 0/228 [00:00<?, ?it/s]

Error at index 8
Error at index 11
Error at index 14
Error w/ sample English Question      How many satellites Saturn has
Bengali Question                   শনির উপগ্রহ কয়টি?
A                                                 63
B                                                 34
C                                                 27
D                                                 13
A Bn                                              63
B Bn                                              34
C Bn                                              27
D Bn                                              13
Correct Answer                                     *
predicted_option                                None
correct_option_fmt                              None
Name: 34, dtype: object
Error at index 37
Error at index 38
Error at index 49
Error at index 55
Error at index 58
Error at index 63
Error at index 77
Error at index 95
Error at index 112
Error at index 119
Error at index 124
Error at index 14

  0%|          | 0/396 [00:00<?, ?it/s]

Error at index 13
Error at index 19
Error at index 23
Error at index 24
Error at index 25
Error at index 31
Error at index 38
Error at index 41
Error at index 45
Error at index 48
Error at index 60
Error at index 63
Error at index 69
Error at index 71
Error at index 73
Error at index 77
Error at index 85
Error at index 92
Error at index 96
Error at index 102
Error at index 107
Error at index 112
Error at index 117
Error at index 121
Error at index 123
Error at index 126
Error at index 128
Error at index 130
Error at index 133
Error at index 136
Error at index 139
Error at index 145
Error at index 150
Error at index 152
Error at index 153
Error at index 155
Error at index 159
Error at index 171
Error at index 174
Error at index 179
Error at index 183
Error at index 190
Error at index 198
Error at index 200
Error at index 206
Error at index 209
Error at index 216
Error at index 220
Error at index 223
Error at index 234
Error at index 235
Error at index 239
Error at index 243
Error at ind

  0%|          | 0/391 [00:00<?, ?it/s]

Error at index 5
Error at index 23
Error at index 32
Error at index 36
Error at index 40
Error at index 59
Error at index 67
Error at index 68
Error at index 119
Error at index 124
Error at index 127
Error at index 129
Error at index 132
Error at index 134
Error at index 136
Error at index 137
Error at index 138
Error at index 140
Error at index 148
Error at index 156
Error at index 158
Error at index 165
Error at index 167
Error at index 168
Error at index 170
Error at index 178
Error at index 181
Error at index 190
Error at index 199
Error at index 213
Error at index 217
Error at index 227
Error at index 230
Error at index 232
Error at index 237
Error at index 239
Error at index 244
Error at index 272
Error at index 280
Error at index 281
Error at index 285
Error at index 291
Error at index 292
Error at index 307
Error at index 310
Error at index 311
Error at index 318
Error at index 327
Error at index 334
Error at index 340
Error at index 343
Error at index 347
Error at index 352
Er

  0%|          | 0/304 [00:00<?, ?it/s]

Error at index 7
Error at index 11
Error at index 14
Error at index 21
Error at index 26
Error at index 34
Error at index 39
Error at index 51
Error at index 53
Error at index 67
Error at index 70
Error at index 78
Error at index 79
Error at index 94
Error at index 96
Error at index 107
Error at index 116
Error at index 123
Error at index 129
Error at index 135
Error at index 142
Error at index 151
Error at index 156
Error at index 168
Error at index 177
Error at index 180
Error at index 189
Error at index 190
Error at index 197
Error at index 202
Error at index 213
Error at index 216
Error at index 229
Error at index 240
Error at index 246
Error at index 248
Error at index 262
Error at index 273
Error at index 286
Error at index 288
Error at index 291
Error at index 298
                                    English Question  \
0  A spring is expanded 10cm by appling 500N forc...   
1  Which one is the angular velocity of the natio...   
2  The maximum velocity and maximum acceleration .

In [55]:
file_index = 4
print(file_paths[file_index])
df = pd.read_csv(file_paths[file_index])

print(df.head())

zs_results = prepare_zs_results(df, dataset_tag = file_tags[file_index])
print(zs_results.head())

https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/12th-Physics-II.csv
                                    English Question  \
0  The ratio of amplitude of two co-herent source...   
1  The maximum magnitude of electric field of an ...   
2  A carnot engine when remain at 72\degree C sin...   
3  A carnot engine when remain at 72\degree C sin...   
4  The phase difference between two points of an ...   

                                    Bengali Question  \
0  দুটি সুসংগত উৎসের বিস্তারের অনুপাত 3 : 2 । তরঙ...   
1  একটি তড়িৎচুম্বকীয় তরঙ্গের তড়িৎ ক্ষেত্রের সর্বো...   
2  একটি কার্নো ইঞ্জিন যখন 72\degree C তাপমাত্রায় ...   
3  একটি কার্নো ইঞ্জিন যখন 72\degree C তাপমাত্রায় ...   
4  একটি তড়িৎ চুম্বকীয় তরঙ্গের উপর অবস্থিত যেকোনো ...   

                                  A                           B  \
0                             9 : 4                       4 : 9   
1  1.62 \times 10^{10}_{1} Wbm^{-2}  1.8 \times 10^{7} Wbm^{-2}   
2                             138 K        

  0%|          | 0/333 [00:00<?, ?it/s]

Error at index 0
Error at index 1
Error at index 6
Error at index 8
Error at index 10
Error at index 14
Error at index 15
Error at index 21
Error at index 22
Error at index 26
Error at index 41
Error at index 50
Error at index 58
Error at index 63
Error at index 64
Error at index 65
Error at index 69
Error at index 74
Error at index 75
Error at index 76
Error at index 78
Error at index 88
Error at index 92
Error at index 93
Error at index 98
Error at index 108
Error at index 128
Error at index 136
Error at index 144
Error at index 148
Error at index 150
Error at index 162
Error at index 164
Error at index 173
Error at index 175
Error at index 179
Error at index 199
Error at index 220
Error at index 221
Error at index 234
Error at index 246
Error at index 254
Error at index 256
Error at index 258
Error at index 271
Error at index 272
Error at index 293
Error at index 297
Error at index 323
Error at index 332
                                    English Question  \
0  The ratio of amplitu

In [56]:
import time
from tqdm.notebook import tqdm

def prepare_ts_results(df, dataset_tag):
    # Assuming df is the original DataFrame and category_translation is defined
    df['predicted_option'] = None
    df['correct_option_fmt'] = None

    # Try to load the progress from the Parquet file
    try:
        df_progress = pd.read_parquet(f"gemini_ts_rqa_{dataset_tag}_results.parquet")
        last_valid_index = df_progress['predicted_option'].last_valid_index()
        start_index = last_valid_index + 1 if last_valid_index is not None else 0
    except (FileNotFoundError, ValueError):
        df_progress = df.copy()
        start_index = 0


    for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
        
        try:
            prompt_input, cor_opt = prepare_questions(row, base_prompt = three_shot, prompt_type = 'three_shot')
        except:
            print("Error w/ sample", row)

        try:
            df.at[i, 'predicted_option'] = get_gemini_output(prompt_input, max_out_tokens = 256)
            df.at[i, 'correct_option_fmt'] = cor_opt
        except Exception as e:
            print(f"Error at index {i}")
            time.sleep(5)

        if (i + 1) % 100 == 0:
            df.to_parquet(f"gemini_ts_rqa_{dataset_tag}_results.parquet")

    # Save final progress
    df.to_parquet(f"gemini_ts_rqa_{dataset_tag}_results.parquet")
    return df

### set file index

for file_index in range(5):
    print(file_paths[file_index])
    df = pd.read_csv(file_paths[file_index])

    print(df.head())

    zs_results = prepare_ts_results(df, dataset_tag = file_tags[file_index])
    print(zs_results.head())

https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/8th-Science.csv
                                   English Question  \
0                   Which acid present in tamarind?   
1               By which two elements fuse is made?   
2  Which on is the tertiary consumer of sundarbans?   
3           Which hormone is sybtgesuzed in leaves?   
4                 Which one is used as insecticide?   

                               Bengali Question              A             B  \
0                        তেঁতুলে কোন এসিড থাকে?          Malic        Citric   
1  কোন মৌল দুটির সাহায্যে ফিউজ প্রস্তুত করা হয়?  Lead and Zinc  Zinc and Tin   
2          সুন্দরবনের তৃতীয় স্তরের খাদক কোনটি?            Pig          Deer   
3                     পাতায় উৎপন্ন হরমোন কোনটি?          Auxin      Ethylene   
4            ফসলের ক্ষতিকর পেস্ট দমন করে কোনটি?       NH_{4}OH          NaOH   

              C              D          A Bn         B Bn        C Bn  \
0        Oxalic       Tartaric       

  0%|          | 0/228 [00:00<?, ?it/s]

Error at index 14
Error w/ sample English Question      How many satellites Saturn has
Bengali Question                   শনির উপগ্রহ কয়টি?
A                                                 63
B                                                 34
C                                                 27
D                                                 13
A Bn                                              63
B Bn                                              34
C Bn                                              27
D Bn                                              13
Correct Answer                                     *
predicted_option                                None
correct_option_fmt                              None
Name: 34, dtype: object
Error at index 37
Error at index 49
Error at index 55
Error at index 58
Error at index 65
Error at index 77
Error at index 95
Error at index 109
Error at index 119
Error at index 161
Error at index 164
Error at index 174
Error w/ sample English Question

  0%|          | 0/396 [00:00<?, ?it/s]

Error at index 13
Error at index 19
Error at index 23
Error at index 31
Error at index 45
Error at index 63
Error at index 71
Error at index 73
Error at index 77
Error at index 96
Error at index 112
Error at index 123
Error at index 126
Error at index 128
Error at index 130
Error at index 139
Error at index 145
Error at index 152
Error at index 153
Error at index 155
Error at index 159
Error at index 171
Error at index 174
Error at index 179
Error at index 183
Error at index 212
Error at index 216
Error at index 220
Error at index 223
Error at index 229
Error at index 234
Error at index 243
Error at index 253
Error at index 267
Error at index 268
Error at index 275
Error at index 277
Error at index 281
Error at index 285
Error at index 298
Error at index 315
Error at index 319
Error at index 326
Error at index 330
Error at index 331
Error at index 334
Error at index 349
Error at index 358
Error at index 363
Error at index 367
Error at index 379
Error at index 385
Error at index 389
Err

...

Error at index 23
Error at index 36
Error at index 40
Error at index 59
Error at index 67
Error at index 127
Error at index 129
Error at index 137
Error at index 140
Error at index 148
Error at index 156
Error at index 158
Error at index 162
Error at index 165
Error at index 167
Error at index 168
Error at index 170
Error at index 173
Error at index 181
Error at index 187
Error at index 199
Error at index 213
Error at index 217
Error at index 222
Error at index 227
Error at index 232
Error at index 237
Error at index 272
Error at index 280
Error at index 281
Error at index 291
Error at index 292
Error at index 297
Error at index 302
Error at index 307
Error at index 310
Error at index 327
Error at index 340
Error at index 343
Error at index 347
Error at index 357
Error at index 358
Error at index 361
Error at index 362
Error at index 382
                                    English Question  \
0  An arrow throws into a wall losses half its ve...   
1              What is the value of \s

  0%|          | 0/304 [00:00<?, ?it/s]

Error at index 7
Error at index 11
Error at index 14
Error at index 21
Error at index 26
Error at index 34
Error at index 51
Error at index 53
Error at index 60
Error at index 67
Error at index 78
Error at index 79
Error at index 86
Error at index 94
Error at index 96
Error at index 98
Error at index 116
Error at index 123
Error at index 129
Error at index 135
Error at index 142
Error at index 151
Error at index 156
Error at index 162
Error at index 168
Error at index 171
Error at index 177
Error at index 180
Error at index 190
Error at index 197
Error at index 202
Error at index 213
Error at index 216
Error at index 229
Error at index 240
Error at index 246
Error at index 248
Error at index 273
Error at index 288
Error at index 291
Error at index 298
                                    English Question  \
0  A spring is expanded 10cm by appling 500N forc...   
1  Which one is the angular velocity of the natio...   
2  The maximum velocity and maximum acceleration ...   
3  The maximum

  0%|          | 0/333 [00:00<?, ?it/s]

Error at index 0
Error at index 1
Error at index 6
Error at index 8
Error at index 10
Error at index 14
Error at index 22
Error at index 26
Error at index 41
Error at index 50
Error at index 63
Error at index 64
Error at index 65
Error at index 74
Error at index 75
Error at index 76
Error at index 78
Error at index 88
Error at index 92
Error at index 98
Error at index 107
Error at index 136
Error at index 148
Error at index 162
Error at index 164
Error at index 173
Error at index 175
Error at index 179
Error at index 199
Error at index 220
Error at index 221
Error at index 234
Error at index 254
Error at index 256
Error at index 258
Error at index 272
Error at index 323
                                    English Question  \
0  The ratio of amplitude of two co-herent source...   
1  The maximum magnitude of electric field of an ...   
2  A carnot engine when remain at 72\degree C sin...   
3  A carnot engine when remain at 72\degree C sin...   
4  The phase difference between two point

In [ ]:
import time
from tqdm.notebook import tqdm

def prepare_cot_results(df, dataset_tag):
    # Assuming df is the original DataFrame and category_translation is defined
    df['predicted_option'] = None
    df['correct_option_fmt'] = None

    # Try to load the progress from the Parquet file
    try:
        df_progress = pd.read_parquet(f"gemini_cot_rqa_{dataset_tag}_results.parquet")
        last_valid_index = df_progress['predicted_option'].last_valid_index()
        start_index = last_valid_index + 1 if last_valid_index is not None else 0
    except (FileNotFoundError, ValueError):
        df_progress = df.copy()
        start_index = 0


    for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
        
        try:
            prompt_input, cor_opt = prepare_questions(row, base_prompt = chain_of_thought, prompt_type = 'cot')
        except:
            print("Error w/ sample", row)

        try:
            df.at[i, 'predicted_option'] = get_gemini_output(prompt_input, max_out_tokens = 1024)
            df.at[i, 'correct_option_fmt'] = cor_opt
        except Exception as e:
            print(f"Error at index {i}")
            time.sleep(5)

        if (i + 1) % 100 == 0:
            df.to_parquet(f"gemini_cot_rqa_{dataset_tag}_results.parquet")

    # Save final progress
    df.to_parquet(f"gemini_cot_rqa_{dataset_tag}_results.parquet")
    return df

### set file index

for file_index in range(5):
    print(file_paths[file_index])
    df = pd.read_csv(file_paths[file_index])

    print(df.head())

    cot_results = prepare_cot_results(df, dataset_tag = file_tags[file_index])
    print(cot_results.head())

https://github.com/sheikhshafayat/BEnQA/raw/master/data/BEnQA/8th-Science.csv
                                   English Question  \
0                   Which acid present in tamarind?   
1               By which two elements fuse is made?   
2  Which on is the tertiary consumer of sundarbans?   
3           Which hormone is sybtgesuzed in leaves?   
4                 Which one is used as insecticide?   

                               Bengali Question              A             B  \
0                        তেঁতুলে কোন এসিড থাকে?          Malic        Citric   
1  কোন মৌল দুটির সাহায্যে ফিউজ প্রস্তুত করা হয়?  Lead and Zinc  Zinc and Tin   
2          সুন্দরবনের তৃতীয় স্তরের খাদক কোনটি?            Pig          Deer   
3                     পাতায় উৎপন্ন হরমোন কোনটি?          Auxin      Ethylene   
4            ফসলের ক্ষতিকর পেস্ট দমন করে কোনটি?       NH_{4}OH          NaOH   

              C              D          A Bn         B Bn        C Bn  \
0        Oxalic       Tartaric       

  0%|          | 0/228 [00:00<?, ?it/s]

Error at index 4
Error at index 8
Error at index 11
Error at index 14
Error w/ sample English Question      How many satellites Saturn has
Bengali Question                   শনির উপগ্রহ কয়টি?
A                                                 63
B                                                 34
C                                                 27
D                                                 13
A Bn                                              63
B Bn                                              34
C Bn                                              27
D Bn                                              13
Correct Answer                                     *
predicted_option                                None
correct_option_fmt                              None
Name: 34, dtype: object
Error at index 37
Error at index 38
Error at index 43
Error at index 49
Error at index 55
Error at index 58
Error at index 65
Error at index 77
Error at index 95
Error at index 104
Error at index 112
E

  0%|          | 0/396 [00:00<?, ?it/s]

Error at index 7
Error at index 13
Error at index 23
Error at index 24
Error at index 25
Error at index 26
Error at index 31
Error at index 41
Error at index 45
Error at index 63
Error at index 69
Error at index 71
Error at index 73
Error at index 85
Error at index 93
Error at index 96
Error at index 102
Error at index 105
Error at index 112
Error at index 117
Error at index 123
Error at index 126
Error at index 128
Error at index 133
Error at index 135
Error at index 144
Error at index 145
Error at index 152
Error at index 153
Error at index 159
Error at index 166
Error at index 169
Error at index 171
Error at index 174
Error at index 179
Error at index 182
Error at index 183
Error at index 188
Error at index 190
Error at index 198
Error at index 200
Error at index 205
Error at index 206
Error at index 209
Error at index 212
Error at index 216
Error at index 220
Error at index 221
Error at index 223
Error at index 229
Error at index 232
Error at index 234
Error at index 235
Error at i

  0%|          | 0/304 [00:00<?, ?it/s]

Error at index 7
Error at index 11
Error at index 14
Error at index 18
Error at index 21
Error at index 26
Error at index 34
Error at index 39
Error at index 46
Error at index 53
Error at index 60
Error at index 67
Error at index 76
Error at index 78
Error at index 94
Error at index 96
Error at index 98
Error at index 122
Error at index 123
Error at index 135
Error at index 142
Error at index 151
Error at index 156
Error at index 157
Error at index 168
Error at index 171
Error at index 177
Error at index 180
Error at index 189
Error at index 190
Error at index 197
Error at index 202
Error at index 213
Error at index 216
Error at index 219
Error at index 228
Error at index 229
Error at index 240
Error at index 246
Error at index 248
Error at index 288
Error at index 291
Error at index 298
                                    English Question  \
0  A spring is expanded 10cm by appling 500N forc...   
1  Which one is the angular velocity of the natio...   
2  The maximum velocity and maxim

  0%|          | 0/333 [00:00<?, ?it/s]

Error at index 0
Error at index 1
Error at index 8
Error at index 14
Error at index 15
Error at index 21
Error at index 22
Error at index 26
Error at index 41
Error at index 63
Error at index 64
Error at index 65
Error at index 66
Error at index 74
Error at index 75
Error at index 76
Error at index 78
Error at index 88
Error at index 90
Error at index 92
Error at index 98
Error at index 107
Error at index 110
Error at index 136
Error at index 144
Error at index 148
Error at index 162
Error at index 164
Error at index 173
Error at index 175
Error at index 176
Error at index 178
Error at index 179
Error at index 188
Error at index 199
Error at index 201
Error at index 220
Error at index 234
Error at index 246
Error at index 254
Error at index 256
Error at index 258
Error at index 272
Error at index 293
Error at index 297
Error at index 323
                                    English Question  \
0  The ratio of amplitude of two co-herent source...   
1  The maximum magnitude of electric f

In [38]:
def clean_predicted_category(text, category_translation):
    # Check if the text is None
    if text is None:
        return ""
    
    # Check if any category is directly present in the prediction string
    for category in category_translation.values():
        if category in text:
            return category
    
    # If no direct match, extract the text within ** ** and remove spaces
    import re
    match = re.search(r'\*\*([^*]*)\*\*', text)
    if match:
        cleaned_text = match.group(1).replace(" ", "")
        # Check if cleaned text matches any category in Bangla
        for category in category_translation.values():
            if cleaned_text == category:
                return category
    
    # If no match found, return None
    return None

In [39]:
clean_predicted_category("এ ই নিবন্ধটি শিক্ষা বিভাগের অন্ত র্ভুক্ত। \n\n...", category_translation)

'শিক্ষা'

In [40]:
clean_predicted_category("প্র েডিক্টেড ক্যাটাগরি: **খে লা** \n\nএই নিবন্.....", category_translation)

'খেলা'

In [41]:
clean_predicted_category(None, category_translation)

''

In [48]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer


def evaluate_classification(df):
    # Map English category to Bangla
    df['category_bn'] = df['category'].map(category_translation)

    # Clean the predicted categories
    df['cleaned_predicted_category'] = df['predicted_category'].apply(clean_predicted_category, args=(category_translation,))

    # Replace None values with an incorrect label to be treated as wrong predictions
    df['cleaned_predicted_category'] = df['cleaned_predicted_category'].fillna('wrong_prediction')

    # Calculate metrics
    y_true = df['category_bn']
    y_pred = df['cleaned_predicted_category']

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    # For AUC-ROC and Log Loss, we need to binarize the labels
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    # Not useful in this setup
    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized, average='macro', multi_class='ovr')
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics

In [50]:
zs_df = pd.read_parquet("gemini_zs_classification_results.parquet")
metrics = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.6848
precision: 0.7057
recall: 0.6164
f1_score: 0.6170
auc_roc: 0.8235
log_loss: 10.9490


/opt/conda/envs/pytorch/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


In [52]:
ts_df = pd.read_parquet("gemini_ts_classification_results.parquet")
metrics = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.6990
precision: 0.6433
recall: 0.5719
f1_score: 0.5761
auc_roc: 0.8313
log_loss: 10.5076


/opt/conda/envs/pytorch/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


In [53]:
cot_df = pd.read_parquet("gemini_cot_classification_results.parquet")
metrics = evaluate_classification(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.5737
precision: 0.6070
recall: 0.4694
f1_score: 0.4880
auc_roc: 0.7626
log_loss: 14.0649


/opt/conda/envs/pytorch/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
